In [1]:
#Importar Librerías

import pandas as pd
import numpy as np

In [2]:
#d1 = pd.read_csv('grd_2023.csv', delimiter = '|')
#d1.info() #Los usé para ver que todo estuviera Ok
#Los usé para ver que todo estuviera Ok, lo dejo por si quieres ver también

#Dataset GRD: Carga y descripción
df = pd.read_parquet('grd_2023_saved.parquet')
print(df.head(3), '\n \n')
df.info()
print('\n \n', df.describe())

   COD_HOSPITAL  CIP_ENCRIPTADO    SEXO FECHA_NACIMIENTO  \
0        122100      72196813.0  HOMBRE       1980-04-13   
1        114101             NaN  HOMBRE       2001-11-04   
2        114101      97627974.0  HOMBRE       2020-07-30   

             SERVICIO_SALUD TIPO_INGRESO FECHA_INGRESO   FECHAALTA  \
0                  VALDIVIA     URGENCIA    2023-03-12  2023-03-14   
1  METROPOLITANO SURORIENTE     URGENCIA    2023-11-04  2023-11-10   
2  METROPOLITANO SURORIENTE     URGENCIA    2023-06-07  2023-06-08   

                       TIPOALTA DIAGNOSTICO1  IR_29301_COD_GRD  IR_29301_PESO  \
0                     DOMICILIO        K35.3           64161.0         0.7094   
1                     DOMICILIO        J93.1           41301.0         1.0869   
2  HOSPITALIZACIÓN DOMICILIARIA        J12.1           44161.0         0.4084   

   IR_29301_SEVERIDAD  IR_29301_MORTALIDAD  EDAD  DIAS_ESTANCIA  
0                 1.0                  1.0  43.0              2  
1                 1.0

In [3]:
#Dataset Hospitales y Procedimientos
hospital = pd.read_parquet('hospital.parquet')
hospital.columns = ['COD_HOSPITAL','HOSPITAL']


ir = pd.read_parquet("ir-grd.parquet")
ir.columns = ['IR_29301_COD_GRD', 'PROCEDIMIENTOS']

cie = pd.read_excel('cie.xlsx')
cie1 = cie[['Código', 'Descripción']]
cie1.columns = ['DIAGNOSTICO1','DESCRIPCION']

#Acá agrego la primera fila porque se iba cortada al cambiar el nombre a las columnas.
hospital.loc[-1] = [101100, 'Hospital Dr. Juan Noé Crevanni (Arica)']       
hospital.index = hospital.index + 1
hospital = hospital.sort_index()

ir.loc[-1] = [11011, 'PH PROCEDIMIENTOS VASCULARES INTRACRANEALES']
ir.index = ir.index + 1
ir = ir.sort_index()


##PARA VER LA INFORMACIÓN.
#print(hospital.head(3),'\n \n')
#hospital.info() 
#print(ir.tail(3), '\n \n')
#ir.info()

In [12]:
#Se cuadra los nombres de hospitales y de procedimientos realizados

grd = pd.merge(df, hospital,on='COD_HOSPITAL',how='left')
grd = pd.merge(grd, ir, on = 'IR_29301_COD_GRD', how = 'left')
grd = pd.merge(grd, cie1, on = 'DIAGNOSTICO1', how = 'left')
print(grd.head(3), '\n \n')
grd.info()
print('\n \n',grd.describe())

   COD_HOSPITAL  CIP_ENCRIPTADO    SEXO FECHA_NACIMIENTO  \
0        122100      72196813.0  HOMBRE       1980-04-13   
1        114101             NaN  HOMBRE       2001-11-04   
2        114101      97627974.0  HOMBRE       2020-07-30   

             SERVICIO_SALUD TIPO_INGRESO FECHA_INGRESO   FECHAALTA  \
0                  VALDIVIA     URGENCIA    2023-03-12  2023-03-14   
1  METROPOLITANO SURORIENTE     URGENCIA    2023-11-04  2023-11-10   
2  METROPOLITANO SURORIENTE     URGENCIA    2023-06-07  2023-06-08   

                       TIPOALTA DIAGNOSTICO1  IR_29301_COD_GRD  IR_29301_PESO  \
0                     DOMICILIO        K35.3           64161.0         0.7094   
1                     DOMICILIO        J93.1           41301.0         1.0869   
2  HOSPITALIZACIÓN DOMICILIARIA        J12.1           44161.0         0.4084   

   IR_29301_SEVERIDAD  IR_29301_MORTALIDAD  EDAD  DIAS_ESTANCIA  \
0                 1.0                  1.0  43.0              2   
1                 1

In [6]:
#Seleccion de Variables de interés, de ellas, solo de una se puede obtener media, mediana, etc.
variables = ['DIAS_ESTANCIA','IR_29301_PESO','HOSPITAL','EDAD', 'IR_29301_MORTALIDAD']
variables1 = ['DIAS_ESTANCIA','IR_29301_PESO','EDAD', 'IR_29301_MORTALIDAD']
grd1 = grd[variables]

#Estadísticas de variables
for variable in variables1:
    print(f"\nEstadísticas para {variable}:")
    print(f"Media: {grd1[variable].mean()}")
    print(f"Mediana: {grd1[variable].median()}")
    print(f"Máximo: {grd1[variable].max()}")
    print(f"Mínimo: {grd1[variable].min()}")
    print(f"Desviación Estandar: {grd1[variable].std()}")


Estadísticas para DIAS_ESTANCIA:
Media: 5.796219075459774
Mediana: 2.0
Máximo: 696
Mínimo: 0
Desviación Estandar: 12.348311332215726

Estadísticas para IR_29301_PESO:
Media: 0.9602886690195916
Mediana: 0.6915
Máximo: 20.6461
Mínimo: 0.0
Desviación Estandar: 1.0908525167664889

Estadísticas para EDAD:
Media: 44.895745096322834
Mediana: 45.0
Máximo: 109.0
Mínimo: 0.0
Desviación Estandar: 25.71346323126398

Estadísticas para IR_29301_MORTALIDAD:
Media: 1.3002201899462944
Mediana: 1.0
Máximo: 3.0
Mínimo: 0.0
Desviación Estandar: 0.9535027277724571


In [7]:
#Estancia media por hospitales
estancia = grd1.groupby('HOSPITAL')['DIAS_ESTANCIA'].mean().reset_index()
print("\nEstancia media por servicio de salud:")
print(estancia.sort_values(by='DIAS_ESTANCIA', ascending=False))

#Peso(costo) medio por hospitales:
peso = grd1.groupby('HOSPITAL')['IR_29301_PESO'].mean().reset_index()
print("\nPeso medio por servicio de salud:")
print(peso.sort_values(by='IR_29301_PESO', ascending=False))


Estancia media por servicio de salud:
                                             HOSPITAL  DIAS_ESTANCIA
21   Hospital Dr. Eduardo Pereira Ramírez (Valpara...      10.046492
64   Instituto Nacional de Enfermedades Respirator...       8.979967
62   Hospital de Urgencia Asistencia Pública Dr. A...       8.952305
27         Hospital Dr. Leonardo Guzmán (Antofagasta)       7.616104
9    Hospital Clínico Metropolitano El Carmen Doct...       7.296645
..                                                ...            ...
39                        Hospital San José (Coronel)       3.955208
53                                   Hospital de Lota       3.634644
65       Instituto Traumatológico Dr. Teodoro Gebauer       3.427290
63                             Hospital de Villarrica       3.105876
60                             Hospital de Santa Cruz       2.977498

[68 rows x 2 columns]

Peso medio por servicio de salud:
                                             HOSPITAL  IR_29301_PESO
64   I

In [9]:
#Columnas a Numpy
age = grd1['EDAD'].to_numpy()
costo = grd1['IR_29301_PESO'].to_numpy()
estancia = grd1['DIAS_ESTANCIA'].to_numpy()
mortal = grd1['IR_29301_MORTALIDAD'].to_numpy()

#Arreglamos datos, se eliminan filas con Nan y se calcula matriz de correlación entre variables.
datos_stack = np.stack([age, costo, estancia, mortal], axis=1)
datos_limpios = datos_stack[~np.isnan(datos_stack).any(axis=1)]
matriz_corr = np.corrcoef(datos_limpios.T)

#Mostramos el resultado
print(matriz_corr)

[[1.         0.10907625 0.06104136 0.10185635]
 [0.10907625 1.         0.46489758 0.40617558]
 [0.06104136 0.46489758 1.         0.34122109]
 [0.10185635 0.40617558 0.34122109 1.        ]]
